In [2]:
import os
import pandas as pd

# ===================== 1️⃣ 路径配置 =====================
paths = {
    "train": "/home/charles/HZU/Data_processed/HSML/MCC5/train/train.csv",
    "val":   "/home/charles/HZU/Data_processed/HSML/MCC5/val/val.csv",
    "test":  "/home/charles/HZU/Data_processed/HSML/MCC5/test/test.csv",
}

# ===================== 2️⃣ 列定义 =====================
AUX_COLUMNS = [
    "speed",
    "torque",
    "gearbox_vibration_x",
    "gearbox_vibration_y",
    "gearbox_vibration_z",
]

LABEL_COL = "class_id"   # ← 如果不是这个名字，改这里

# ===================== 3️⃣ 拆分并保存 =====================
for split, csv_path in paths.items():
    print(f"\n===== Processing {split.upper()} =====")

    df = pd.read_csv(csv_path)

    # ---------- 列检查 ----------
    missing_aux = [c for c in AUX_COLUMNS if c not in df.columns]
    if missing_aux:
        raise ValueError(f"{split} 缺少辅助模态列: {missing_aux}")

    if LABEL_COL not in df.columns:
        raise ValueError(f"{split} 缺少标签列: {LABEL_COL}")

    # ---------- 辅助模态（+ label） ----------
    df_aux = df[AUX_COLUMNS + [LABEL_COL]].copy()

    # ---------- 主模态（去掉辅助模态，但保留 label） ----------
    MAIN_COLUMNS = [c for c in df.columns if c not in AUX_COLUMNS]
    df_main = df[MAIN_COLUMNS].copy()

    # ---------- 输出路径 ----------
    base_dir = os.path.dirname(csv_path)
    aux_path  = os.path.join(base_dir, f"{split}_aux.csv")
    main_path = os.path.join(base_dir, f"{split}_main.csv")

    # ---------- 保存 ----------
    df_aux.to_csv(aux_path, index=False)
    df_main.to_csv(main_path, index=False)

    # ---------- 信息 ----------
    print(f"✅ Main modality saved: {main_path}")
    print(f"   Shape: {df_main.shape}")
    print(f"✅ Aux modality  saved: {aux_path}")
    print(f"   Shape: {df_aux.shape}")



===== Processing TRAIN =====
✅ Main modality saved: /home/charles/HZU/Data_processed/HSML/MCC5/train/train_main.csv
   Shape: (600000, 4)
✅ Aux modality  saved: /home/charles/HZU/Data_processed/HSML/MCC5/train/train_aux.csv
   Shape: (600000, 6)

===== Processing VAL =====
✅ Main modality saved: /home/charles/HZU/Data_processed/HSML/MCC5/val/val_main.csv
   Shape: (100000, 4)
✅ Aux modality  saved: /home/charles/HZU/Data_processed/HSML/MCC5/val/val_aux.csv
   Shape: (100000, 6)

===== Processing TEST =====
✅ Main modality saved: /home/charles/HZU/Data_processed/HSML/MCC5/test/test_main.csv
   Shape: (300000, 4)
✅ Aux modality  saved: /home/charles/HZU/Data_processed/HSML/MCC5/test/test_aux.csv
   Shape: (300000, 6)


In [7]:
import os
import pandas as pd
import numpy as np
import pywt

# ======================================================
# 路径
# ======================================================

paths = {
    "train": "/home/charles/HZU/Data_processed/HSML/MCC5/train/train_main.csv",
    "val":   "/home/charles/HZU/Data_processed/HSML/MCC5/val/val_main.csv",
    "test":  "/home/charles/HZU/Data_processed/HSML/MCC5/test/test_main.csv",
}

# ======================================================
# 主模态振动通道（只用 motor vibration）
# ======================================================

VIB_CHANNELS = [
    "motor_vibration_x",
    "motor_vibration_y",
    "motor_vibration_z",
]

LABEL_COL = "class_id"

# ======================================================
# 参数
# ======================================================

WINDOW = 512
STEP   = 256

WIDTHS  = np.arange(1, 32)
WAVELET = "morl"

# ======================================================
# Z-score（样本内标准化）
# ======================================================

def normalize_signal(sig):
    mean = sig.mean(axis=0, keepdims=True)
    std  = sig.std(axis=0, keepdims=True) + 1e-8
    return (sig - mean) / std

# ======================================================
# 主处理函数（方案 A）
# ======================================================

def process_csv(split_name, csv_path):

    print(f"\n==================== {split_name.upper()} ====================")
    print("Loading:", csv_path)

    df = pd.read_csv(csv_path)

    # ---------- 列检查 ----------
    for c in VIB_CHANNELS + [LABEL_COL]:
        if c not in df.columns:
            raise ValueError(f"Missing column: {c}")

    X_raw = df[VIB_CHANNELS].values        # [N, 3]
    y     = df[LABEL_COL].values           # [N]

    N = len(df)
    print("Total samples:", N)

    # ---------- 振动归一化 ----------
    X = normalize_signal(X_raw)

    cwt_rows       = []
    window_labels  = []
    window_starts  = []

    # ---------- 滑窗 + CWT ----------
    for start in range(0, N - WINDOW + 1, STEP):

        chunk = X[start:start + WINDOW]           # [WINDOW, 3]
        labels_chunk = y[start:start + WINDOW]

        # 众数标签
        u, c = np.unique(labels_chunk, return_counts=True)
        win_label = u[np.argmax(c)]

        win_feat = []

        for ch in range(len(VIB_CHANNELS)):
            sig = chunk[:, ch]                    # [WINDOW]

            cwt_mat, _ = pywt.cwt(sig, WIDTHS, WAVELET)
            cwt_abs = np.abs(cwt_mat)             # [S, WINDOW]

            # scale 维度平均
            cwt_mean = cwt_abs.mean(axis=1)       # [S]
            win_feat.extend(cwt_mean.tolist())

        cwt_rows.append(win_feat)
        window_labels.append(win_label)
        window_starts.append(start)

    # ---------- 输出 ----------
    cwt_rows = np.asarray(cwt_rows)
    scale_dim = len(WIDTHS)

    print("Windows:", len(cwt_rows))
    print("Feature dim:", cwt_rows.shape[1])

    # 列名
    cols = []
    for ch in range(len(VIB_CHANNELS)):
        for s in range(scale_dim):
            cols.append(f"ch{ch}_scale{s}")

    df_out = pd.DataFrame(cwt_rows, columns=cols)
    df_out.insert(0, "window_start", window_starts)
    df_out["label"] = window_labels

    save_path = os.path.join(
        os.path.dirname(csv_path),
        f"{split_name}_CWT_main.csv"
    )
    df_out.to_csv(save_path, index=False)

    print("Saved:", save_path)

# ======================================================
# 执行
# ======================================================

for name, path in paths.items():
    process_csv(name, path)

print("\n🎉 主模态 CWT（方案 A）处理完成")



==================== TRAIN ====================
Loading: /home/charles/HZU/Data_processed/HSML/MCC5/train/train_main.csv
Total samples: 600000
Windows: 2342
Feature dim: 93
Saved: /home/charles/HZU/Data_processed/HSML/MCC5/train/train_CWT_main.csv

==================== VAL ====================
Loading: /home/charles/HZU/Data_processed/HSML/MCC5/val/val_main.csv
Total samples: 100000
Windows: 389
Feature dim: 93
Saved: /home/charles/HZU/Data_processed/HSML/MCC5/val/val_CWT_main.csv

==================== TEST ====================
Loading: /home/charles/HZU/Data_processed/HSML/MCC5/test/test_main.csv
Total samples: 300000
Windows: 1170
Feature dim: 93
Saved: /home/charles/HZU/Data_processed/HSML/MCC5/test/test_CWT_main.csv

🎉 主模态 CWT（方案 A）处理完成
